In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
data=pd.read_csv('/content/merged_data.csv')

In [3]:
# Séparation des fonctionnalités et de la variable cible
X = data.drop(columns=["label"])
y = data["label"]

In [4]:
# Sélectionner uniquement les colonnes numériques
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
X_numeric = X[numeric_cols]


In [5]:

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_numeric , y, test_size=0.2, random_state=42)


In [6]:

# Créer un pipeline avec la normalisation, la sélection de caractéristiques et le modèle
def create_pipeline(model):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif)),
        ('model', model)
    ])
    return pipeline


In [7]:
# Définir les modèles à tester
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression()
}

In [8]:
# Définir les paramètres pour la sélection de caractéristiques
selector_params = {'selector__k': [5, 10, 15,20,23]}  # Nombre de caractéristiques à sélectionner

# Créer un objet de validation croisée
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [9]:
# Pour chaque modèle, évaluez sa performance avec la validation croisée
for name, model in models.items():
    pipeline = create_pipeline(model)

    # Recherche sur grille avec validation croisée
    grid_search = GridSearchCV(pipeline, param_grid=selector_params, cv=kfold, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Obtenir le meilleur score de validation croisée et les meilleurs paramètres
    best_score = grid_search.best_score_
    best_params = grid_search.best_params_

    # Évaluer le modèle sur l'ensemble de test
    y_pred = grid_search.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    # Afficher les résultats
    print(f"Modèle: {name}")
    print(f"Meilleur score de validation croisée: {best_score}")
    print(f"Meilleurs paramètres de sélection de caractéristiques: {best_params}")
    print(f"Précision sur l'ensemble de test: {test_accuracy}")
    print("---------------------------------------")



Modèle: SVM
Meilleur score de validation croisée: 0.9521335618963356
Meilleurs paramètres de sélection de caractéristiques: {'selector__k': 20}
Précision sur l'ensemble de test: 0.954574951330305
---------------------------------------
Modèle: Random Forest
Meilleur score de validation croisée: 0.9579753478476107
Meilleurs paramètres de sélection de caractéristiques: {'selector__k': 23}
Précision sur l'ensemble de test: 0.9558728098637248
---------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modèle: Logistic Regression
Meilleur score de validation croisée: 0.9269829420376865
Meilleurs paramètres de sélection de caractéristiques: {'selector__k': 23}
Précision sur l'ensemble de test: 0.9260220635950681
---------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
from sklearn.metrics import classification_report

# Entraîner le modèle Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = random_forest.predict(X_test)

# Évaluer les prédictions
print("Rapport de classification:")
print(classification_report(y_test, y_pred))


Rapport de classification:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      1384
         1.0       0.89      0.64      0.75       157

    accuracy                           0.96      1541
   macro avg       0.92      0.82      0.86      1541
weighted avg       0.95      0.96      0.95      1541



In [11]:
# Afficher les prédictions associées à chaque exemple dans l'ensemble de test
print("Prédictions sur l'ensemble de test:")
for i in range(len(X_test)):
    print("Exemple {}: Valeur réelle = {}, Prédiction = {}".format(i+1, y_test.iloc[i], y_pred[i]))



Prédictions sur l'ensemble de test:
Exemple 1: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 2: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 3: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 4: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 5: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 6: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 7: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 8: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 9: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 10: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 11: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 12: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 13: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 14: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 15: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 16: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 17: Valeur réelle = 1.0, Prédiction = 0.0
Exemple 18: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 19: Valeur réelle = 0.0, Prédiction = 0.0
Exemple 20: Valeur réel